# Fine-tuning a model with the Trainer API or Keras

In [ ]:
# Install the Transformers, Datasets, and Evaluate libraries to run this notebook.
!pip install datasets evaluate transformers[sentencepiece]

<small>Transformers предоставляет класс Trainer, который позволяет выполнить  тонкую настройку предварительно обученной модели на нашем наборе данных.</small>

## Подготовка данных

In [29]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

raw_datasets = load_dataset("glue", "mrpc")
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [30]:
def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)

In [31]:
tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)

Map:   0%|          | 0/408 [00:00<?, ? examples/s]

In [ ]:
examples = tokenized_datasets["train"][:5]
for i in range(5):
    print(len(examples["input_ids"][i]))  # 50, 59, 47, 67, 59

# или
# for example in tokenized_datasets["train"].select(range(5)):
#     print(len(example["input_ids"]))

In [32]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
data_collator
# мы его будем использовать ниже, когда будем передавать параметры "тренеру"

## Обучение


<small>Первый шаг перед тем, как мы сможем определить наш Trainer, — это определить класс TrainingArguments, который будет содержать все гиперпараметры, которые Trainer будет использовать для обучения и оценки. Единственный аргумент, который вам нужно предоставить, — это каталог, в котором будет сохранена обученная модель, а также контрольные точки по пути. Для всего остального вы можете оставить значения по умолчанию, которые должны работать довольно хорошо для базовой тонкой настройки.</small>

In [33]:
from transformers import TrainingArguments
# training_args = TrainingArguments("test-trainer")
training_args = TrainingArguments(
    output_dir="./bert-output",
    report_to="none"  # Отключает wandb
)
# training_args

<small>Используем `AutoModelForSequenceClassification` класс моделей, указываем к-во классов для предсказания, создаем  экземпляр модели:

In [ ]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

<small>созданный экземпляр этой предварительно обученной модели - BERT не был обучен классификации пар предложений, поэтому старая "голова" был отброшена, а в новой веса инициализированы случайным образом. В заключение предлагается обучить модель, что мы и собираемся сделать сейчас.  
И мы формируем наш trainer, передав ему — модель, гиперпараметры (training_args), наборы данных для обучения и проверки, наш data_collator и наш tokenizer:</small>

In [34]:
from transformers import Trainer

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    # data_collator=data_collator,
    processing_class=tokenizer,
)

<small>Здесь мы передали токенизатор, который мы использовали, когда ранее формировали data_collator (`data_collator = DataCollatorWithPadding(tokenizer=tokenizer`). По умолчанию, Trainer аналогично формирует data_collator, поэтому можно пропустить строку "data_collator=data_collator" в этом вызове.   
И, чтобы точно настроить модель на нашем наборе данных, осталось вызвать метод train() нашего Trainer:<.small>

In [35]:
# Пооверяем, подключен ли графический ускоритель
# Иначе дообучение будет длится несколько часов (с ускорителем - несколько минут)
import torch
print("GPU доступен:", torch.cuda.is_available())
print("Название GPU:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "Нет GPU")

GPU доступен: True
Название GPU: Tesla T4


In [36]:
trainer.train()

Step,Training Loss
500,0.242400
1000,0.114300


TrainOutput(global_step=1377, training_loss=0.15301497881189152, metrics={'train_runtime': 209.1206, 'train_samples_per_second': 52.62, 'train_steps_per_second': 6.585, 'total_flos': 405114969714960.0, 'train_loss': 0.15301497881189152, 'epoch': 3.0})

<small>Это запустит тонкую настройку (которая должна занять пару минут на GPU) и сообщит о потере обучения каждые 500 шагов. Однако это не отражает, насколько хорошо (или плохо) работает ваша модель, т.к. мы не указали параметры оценки: Evaluation_strategy на "steps" (оценивать каждые eval_steps) или "epoch" (оценивать в конце каждой эпохи). И мы не предоставили Тренеру функцию compute_metrics() для расчета метрики во время указанной оценки (иначе оценка просто вывела бы потерю, что не является очень интуитивно понятным числом).   
Чтобы получить прогнозы используется команда `trainer.predict()`:</small>

In [37]:
predictions = trainer.predict(tokenized_datasets["validation"])
print(predictions.predictions.shape, predictions.label_ids.shape)

(408, 2) (408,)


In [38]:
print(predictions[0][0]) # [-2.6618629  2.3648045]
# или
# print(predictions.predictions[0]) # [-2.6618629  2.3648045]
# так как predictions – это объект, содержащий:
# _________predictions.predictions – массив логитов (сырые выходы нейросети).
# _________predictions.label_ids – истинные метки (если они есть).
# _________predictions.metrics – метрики модели.

# print(predictions[1]) # [1 0 0 1 0 ...]
# print(predictions[2]) # {'test_loss': 0.686876118183136, 'test_runtime': 2.7329, 'test_samples_per_second': 149.294, 'test_steps_per_second': 18.662}

[-3.1932015  3.4147985]


<small>Выход метода predict() — это двумерный массив размером 408 x 2 (408 — количество элементов в наборе данных) в форме кортежа с тремя полями: `predicts, label_ids и metrics`, где  
`predicts[0]` — содержит логиты для каждого элемента набора данных и чтобы преобразовать их в прогнозы, которые можно сравнить с нашими метками, нужно взять индекс с максимальным значением на второй оси:</small>

In [39]:
import numpy as np
preds = np.argmax(predictions.predictions, axis=-1)
preds[:5] # array([1, 0, 0, 1, 0])

array([1, 0, 1, 1, 0])

### __Что здесь происходит__  

в `predictions.predictions` мы имеем логиты вида `[-2.6618629  2.3648045]`  
где, если 1 первое значение больше второго, то у нас предложения не являются парафразами, если иначе - они парафразы. Функция np.argmax() находит индекс наибольшего значения по указанной оси, то есть, если второе значение больше  
'''
Индекс            0           1
Логит       [-2.6618629  __2.3648045__]
'''
то `np.argmax()` вернет индекс 1, что означает (в соответствии с определенными в датасете значениями меток: 1 - парафразы, 0 - нет) предложения являются парафразами и наоборот.

__По поводу оси (axis=-1).__  
<small>Так как `logits` содержит массив логитов размерности:
`(batch_size, num_classes)`, где  
- `batch_size` – количество примеров в батче,  
- `num_classes` – количество классов, для которых модель предсказывает логиты,  
то `axis=1 (или axis=-1)` – ищет максимум по классам - для каждого примера найдёт самый вероятный класс (в то время, как `axis=0` – ищет максимум по батчам - вернёт один максимум на каждый класс).</small>

Еще пример (несколько классов).
```
logits = np.array([
    [2.3, 1.1, 0.5],  # 1-й пример → 2.3 (индекс 0)
    [0.2, 3.4, 2.1],  # 2-й пример → 3.4 (индекс 1)
    [1.2, 0.8, 4.0]   # 3-й пример → 4.0 (индекс 2)
])

preds = np.argmax(logits, axis=-1)
print(preds)  # [0 1 2]
```


Кстати, если вместо предсказанных классов нужны вероятности (например, для анализа уверенности модели), надо использовать softmax:

In [23]:
from scipy.special import softmax

probs = softmax(predictions.predictions, axis=-1)  # Преобразуем логиты в вероятности
print(probs[:5])  # Теперь числа от 0 до 1, сумма по каждому примеру = 1

[[0.00651788 0.99348205]
 [0.9929617  0.00703833]
 [0.06869959 0.93130034]
 [0.00665813 0.99334186]
 [0.99389595 0.00610403]]


### Evaluation (Оценка)

__Теперь можно сравнить эти предсказанные значения с метками.__  
Чтобы построить функцию `compute_metric()`, используем метрики из библиотеки 🤗 `Evaluate`.  
Загрузим метки из набора данных MRPC с помощью функции `estimate.load()` и выполним сравнение с предсказанными метками с применением метода `compute()`, который выдаёт такие метрики, как `accuracy` (точность) и `F1-score`.  
<small>Accuracy – доля правильно предсказанных примеров.
F1-score – баланс между точностью (precision) и полнотой (recall).
</small>

In [40]:
import evaluate

metric = evaluate.load("glue", "mrpc")
metric.compute(predictions=preds, references=predictions.label_ids)

{'accuracy': 0.8627450980392157, 'f1': 0.9041095890410958}

`evaluate.load("glue", "mrpc")` - загружает метрику GLUE (General Language Understanding Evaluation). mrpc (Microsoft Research Paraphrase Corpus) – это задача бинарной классификации: определение, являются ли два предложения перефразами.Выдаёт такие метрики, как accuracy (точность) и F1-score.  
Что здесь - `predictions=preds, references=predictions.label_ids`  
- `predictions=preds` - в preds нули и единицы, т.е., то что мы предсказали  
- `predictions.label_ids` - как было выше отмечено в пояснениях,  


<small>__`predictions`__ – это объект, содержащий:
- `predictions.predictions` – массив логитов (сырые выходы нейросети),  
- `predictions.label_ids` – истинные метки,  
- `predictions.metrics` – метрики модели.</small>

## Объединив все

Объединяя все вместе, мы получаем нашу функцию compute_metrics():

In [42]:
def compute_metrics(eval_preds):
    metric = evaluate.load("glue", "mrpc")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

### Разбор `compute_metrics=compute_metrics`  

Trainer сам её вызовет во время оценки (evaluation), передавая нужные данные.  
Что передаёт Trainer в compute_metrics?  
Во время вызова trainer.evaluate() или по окончании эпохи evaluation_strategy="epoch", Trainer вычисляет предсказания и передаёт их в compute_metrics.  
📌 Аргумент `eval_preds` внутри `compute_metrics(eval_preds)` – это кортеж: `logits, labels = eval_preds`  
Где:  
- `logits` – массив логитов модели (размерность (num_examples, num_labels)).  
- `labels` – истинные метки (размерность (num_examples,)).  

Пример eval_preds:  
```
np.array([[2.1, 0.3], [0.1, 3.2], [1.2, 1.8]]),  # Логиты (num_examples, num_labels)
np.array([0, 1, 1])  # Метки (num_examples,)
```

### Создаем новый TrainingArguments

Создаем новый TrainingArguments с его evaluation_strategy, установленным на "epoch", и новую модель (в противном случае мы бы просто продолжили обучение модели, которую мы уже обучили).

In [ ]:
training_args_2 = TrainingArguments("test-trainer", evaluation_strategy="epoch",  report_to="none")
model_2 = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

trainer_2 = Trainer(
    model_2,
    training_args_2,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,  # Передаём функцию, но не вызываем её!
)

 `"test-trainer"` – это название директории, куда Trainer сохранит файлы (можно поменять название).  
📌 Что хранится в "test-trainer"?  
Когда ты запускаешь trainer.train(), Trainer автоматически создаёт в "test-trainer":  
- Сохранённые веса модели (pytorch_model.bin)  
- Конфиг модели (config.json)  
- Логи тренировки (trainer_state.json)  
- Аргументы обучения (training_args.bin)  
- Tokenizer (если Trainer использует токенизатор)  
- Снимки (checkpoint) модели (если включены сохранения)  

In [44]:
trainer_2.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.371126,0.857843,0.901024
2,0.527200,0.459970,0.845588,0.893401
3,0.279600,0.774146,0.848039,0.894915


TrainOutput(global_step=1377, training_loss=0.3341553964386319, metrics={'train_runtime': 209.3512, 'train_samples_per_second': 52.562, 'train_steps_per_second': 6.577, 'total_flos': 405114969714960.0, 'train_loss': 0.3341553964386319, 'epoch': 3.0})

Здесь тонкая настройка выполнена с использованием API Trainer (см. примеры в гл. 7 курса). В 8 части сделаем то же в чистом PyTorch.

## Как загрузить уже обученную модель?

После обучения, Trainer сохраняет файлы в папке "test-trainer" (или другой, которую ты указал). Теперь мы можем загрузить модель обратно:

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
checkpoint = "test-trainer"  # Путь к сохранённой модели

1. Загружаем модель и токенизатор

In [ ]:
model_load = AutoModelForSequenceClassification.from_pretrained(checkpoint)
tokenizer_load = AutoTokenizer.from_pretrained(checkpoint)

📌 Что здесь происходит?  
`from_pretrained(checkpoint)` - загружает модель из указанной папки (test-trainer).  
- Веса модели (`pytorch_model.bin)`,
- конфиг (`config.json`) и токенизатор загружаются автоматически.  

 2️⃣ Как сделать предсказания?

In [ ]:
# Не очевидный пример, но ...
from transformers import pipeline

classifier = pipeline("text-classification", model=model_load, tokenizer=tokenizer_load)

text = "This is an amazing example!"
result = classifier(text)
print(result)
# Вывод будет примерно таким:
# [{'label': 'LABEL_1', 'score': 0.98}]
# 📌 В зависимости от модели, метки (LABEL_0, LABEL_1)
# могут соответствовать разным классам
#  (например, "not paraphrase" и "paraphrase" в MRPC).



🔄 3️⃣ Как дообучить модель?

In [ ]:
trainer_load = Trainer(
    model=model_load,  # Используем уже обученную модель
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer_load,
    compute_metrics=compute_metrics, # Если мы ее предварительно создали (и под эту модель!)
)

In [ ]:
trainer.train()  # Продолжаем обучение!